# TD0 — Limites OLTP et transition DWH/OLAP

## Objectifs pédagogiques
- Identifier les limites d'une base OLTP pour l'analyse.
- Expliquer pourquoi passer à un modèle décisionnel (DWH/OLAP).
- Esquisser un plan de transition vers un modèle en étoile.

> Durée : 20-30 min. Aucun prérequis de code, seulement lecture et réflexion.

## Diagnostic OLTP

- **Requêtes typiques** : stock produit/magasin, consultation client, suivi commande (SELECT/FROM/WHERE simples).
- **Limites** : agrégations multi-dimensionnelles difficiles (ex. CA par mois/catégorie/région), historique peu exploitable.
- **Performance** : schéma normalisé ⇒ nombreuses jointures pour analyser ⇒ lenteurs.

### Diagramme OLTP actuel (exemple)
```mermaid
erDiagram
  PRODUIT ||--o{ VENTE : "produit_id"
  MAGASIN ||--o{ VENTE : "magasin_id"
  CLIENT ||--o{ VENTE : "client_id"
  VENTE {
    vente_id PK
    produit_id FK
    magasin_id FK
    client_id FK
    quantite
    montant
    date_vente
  }
```

> Questions rapides : (1) quelle requête devient lente ? (2) quelles colonnes manquent pour l'analyse ?

## Exemple SQL OLTP concret (pour montrer la complexité analytique)

Tables courantes : `commandes`, `lignes_commande`, `produits`, `clients`.

```sql
-- CA mensuel par catégorie et ville dans un schéma OLTP normalisé
SELECT
  strftime('%Y-%m', c.date_commande) AS mois,
  p.categorie,
  cl.ville,
  SUM(lc.quantite * lc.prix_unitaire) AS ca
FROM commandes c
JOIN lignes_commande lc ON lc.commande_id = c.commande_id
JOIN produits p ON p.produit_id = lc.produit_id
JOIN clients cl ON cl.client_id = c.client_id
GROUP BY strftime('%Y-%m', c.date_commande), p.categorie, cl.ville
ORDER BY mois, ca DESC;
```

> Problème : 3 jointures et agrégations ⇒ coûteux en OLTP. En DWH/étoile, ce calcul est plus direct et mieux indexé.

## Pourquoi l'OLTP ne suffit pas pour le décisionnel ?

- Analyses attendues : tendance CA, top produits, saisonnalité, panier moyen.
- Problèmes en OLTP : requêtes lourdes, données historiques fragmentées, schéma centré transaction.
- Solution : modèle décisionnel (étoile) avec tables de faits + dimensions pour agréger vite.

In [ ]:
# Démo rapide OLTP vs (pré-)OLAP avec SQLite en mémoire
import sqlite3
import pandas as pd

conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Schéma OLTP minimal
cursor.executescript('''
CREATE TABLE clients (client_id INTEGER PRIMARY KEY, nom TEXT, ville TEXT);
CREATE TABLE produits (produit_id INTEGER PRIMARY KEY, produit_nom TEXT, categorie TEXT);
CREATE TABLE commandes (commande_id INTEGER PRIMARY KEY, client_id INT, date_commande TEXT,
                       FOREIGN KEY(client_id) REFERENCES clients(client_id));
CREATE TABLE lignes_commande (commande_id INT, produit_id INT, quantite INT, prix_unitaire REAL,
                             FOREIGN KEY(commande_id) REFERENCES commandes(commande_id),
                             FOREIGN KEY(produit_id) REFERENCES produits(produit_id));

INSERT INTO clients VALUES (1, 'Alice', 'Paris'), (2, 'Bob', 'Lyon');
INSERT INTO produits VALUES (1, 'Laptop', 'Informatique'), (2, 'Phone', 'Téléphonie');
INSERT INTO commandes VALUES (10, 1, '2024-01-02'), (11, 2, '2024-02-05');
INSERT INTO lignes_commande VALUES
  (10, 1, 1, 1200.0),
  (10, 2, 2, 600.0),
  (11, 2, 1, 650.0);
''')

# Requête OLTP : CA mensuel par catégorie et ville (multi-jointures)
req_oltp = '''
SELECT strftime('%Y-%m', c.date_commande) AS mois, p.categorie, cl.ville,
       SUM(lc.quantite * lc.prix_unitaire) AS ca
FROM commandes c
JOIN lignes_commande lc ON lc.commande_id = c.commande_id
JOIN produits p ON p.produit_id = lc.produit_id
JOIN clients cl ON cl.client_id = c.client_id
GROUP BY strftime('%Y-%m', c.date_commande), p.categorie, cl.ville
ORDER BY mois, ca DESC;
'''
print("OLTP (3 jointures) :")
display(pd.read_sql_query(req_oltp, conn))

# (Pré-)OLAP : table de faits déjà jointes + dimensions simplifiées
cursor.executescript('''
CREATE TABLE fact_ventes AS
SELECT strftime('%Y-%m', c.date_commande) AS mois,
       p.categorie,
       cl.ville,
       lc.quantite * lc.prix_unitaire AS montant
FROM commandes c
JOIN lignes_commande lc ON lc.commande_id = c.commande_id
JOIN produits p ON p.produit_id = lc.produit_id
JOIN clients cl ON cl.client_id = c.client_id;

CREATE INDEX idx_fact_mois_cat_ville ON fact_ventes(mois, categorie, ville);
''')

req_olap = '''
SELECT mois, categorie, ville, SUM(montant) AS ca
FROM fact_ventes
GROUP BY mois, categorie, ville
ORDER BY mois, ca DESC;
'''
print("\n(Pré-)OLAP (agrégat direct, indexable) :")
display(pd.read_sql_query(req_olap, conn))

conn.close()

In [ ]:
# Illustration rapide (lecture seule)
"""
Requête OLTP :
SELECT quantite FROM stock WHERE produit_id='P01' AND magasin_id='M01';

Problème : pour « CA par mois et catégorie », il faut joindre ventes, produits, catégories, dates ⇒ lourd et lent.
Solution : modéliser un fait VENTES avec dimensions PRODUIT, MAGASIN, DATE.
"""

print("Retenez : OLTP optimise la transaction, pas l'analyse.")

## Schéma cible DWH

Transition vers modèle étoile (TD1).

## Cibles DWH/OLAP (rappel de la correction)
- **DWH** : stockage historique intégré, orienté analyse.
- **OLAP** : cube multidimensionnel pour slice/dice/roll-up.
- **Transition** : ETL depuis OLTP vers DWH.

## Plan de passage (3 étapes)
1. **Audit OLTP** : sources, grain, volumétrie, qualité.
2. **Modélisation DWH** : étoile/flocon, dimensions/faits, clés de substitution.
3. **ETL initial** : chargement historique + incrémental.

## Livrables attendus
- Diagnostic écrit : 3 limites OLTP avec exemples.
- Schéma cible : DWH étoile simple (Mermaid ou SVG export).
- Plan transition : 3 étapes avec responsabilités.

## Plan transition

1. Audit sources OLTP.
2. Modélisation DWH.
3. ETL initial.